# Search Engine Experiments

Test of different tokenization methods and Word Embedding for the search engine of the articles of Equinox by Asesoftware

## CSV of Articles

CSV Columns: “article_name”, “enumeration_in_article”, “content” 
“stringWithAllText”

In [1]:
import pandas as pd
df = pd.read_csv("articles_paragraphs.csv")

In [2]:
df_esp = df[df['language_code'] =='es'].reset_index()

In [3]:
df_esp

,index,Unnamed: 0,article_name,content,enumeration_in_article,file_id,language_code
0,13,13,La inteligencia artificial en beneficio a la h...,Todos tenemos una opinión frente a lo que es i...,0,1,es
1,14,14,La inteligencia artificial en beneficio a la h...,"En contraste, dentro de nuestra realidad, pode...",1,1,es
2,15,15,La inteligencia artificial en beneficio a la h...,"Con esto en mente, junto a nuestro nos dimos ...",2,1,es
3,16,16,La inteligencia artificial en beneficio a la h...,"En las ciencias de la computación, la AI es la...",3,1,es
4,17,17,La inteligencia artificial en beneficio a la h...,La inteligencia artificial débil (weak AI o Na...,4,1,es
...,...,...,...,...,...,...,...
190,578,578,Aprendizaje profundo o nadar en la orilla,Dificultad para interpretar cómo funciona el m...,9,22,es
191,579,579,Aprendizaje profundo o nadar en la orilla,Poder de cómputo exigido\r\nParece que ya lo h...,10,22,es
192,580,580,Aprendizaje profundo o nadar en la orilla,"En resumen, ¿Qué se debe tener en cuenta antes...",11,22,es
193,581,581,Aprendizaje profundo o nadar en la orilla,¿ESTÁ TODO? ¡FELICITACIONES! DÉ EL SALTO Y DIS...,12,22,es


## Data Preprocessing and Tokenization

### Lemma Whitespace Tokenization

In [4]:
!python -m spacy download es_core_news_sm

     --------------------------------------- 12.9/12.9 MB 16.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [5]:
import pandas as pd
import string
import spacy

'''
In this example, we use the Spacy library to preprocess and tokenize the text, 
lowercasing the text, removing punctuation, lemmatizing the words, and removing stopwords 
and short words. We then apply this function to each paragraph in the 'content' column of the CSV file using a for loop, 
and append the resulting list of tokens to a list of lists. The final result is a list of lists, where each 
sublist contains the tokens of each paragraph.

'''

# load spacy nlp model
nlp = spacy.load('es_core_news_sm')

# define function for pre-processing and tokenization
def preprocess_text_lemma(text):
    # lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # lemmatize
    doc = nlp(text)
    lemmatized_text = [token.lemma_ for token in doc]
    # remove stopwords and short words
    stopwords = spacy.lang.es.stop_words.STOP_WORDS
    tokens = [token for token in lemmatized_text if token not in stopwords and len(token) > 2]
    return tokens

# apply pre-processing and tokenization to the 'content' column of each row
tokenized_paragraphs_lemma_es = []
for paragraph in df_esp['content']:
    tokens = preprocess_text_lemma(paragraph)
    tokenized_paragraphs_lemma_es.append(tokens)

# print the resulting list of lists of tokens
print(tokenized_paragraphs_lemma_es)


[['opinión', 'frente', 'inteligencia', 'artificial', 'blog', 'llamar', 'cine', 'televisión', 'comunicación', 'masivo', 'empapar', 'historia', 'personaje', 'relacionado', 'construir', 'imaginario', 'frente', 'tema', 'tomar', 'ejemplo', 'universo', 'cinematográfico', 'marvel', 'studio', 'presentar', 'inteligencia', 'artificial', 'visión', 'superhéroir', 'cuyo', 'mente', 'poseer', 'resto', 'jarvis', 'asistente', 'personal', 'virtual', 'tony', 'stark', 'contraparte', 'infame', 'ultrón', 'robot', 'cuyo', 'objetivo', 'exterminar', 'raza', 'humano'], ['contraste', 'realidad', 'pensar', 'sistema', 'siri', 'alexa', 'presente', 'primordialmente', 'celular', 'asistirno', 'controversial', 'cámara', 'reconocimiento', 'facial', 'implementada', 'china', 'cuyo', 'generar', 'revuelta', 'hong', 'kong', 'sentir', 'tipo', 'tecnología', 'llegar', 'vulnerar', 'privacidad', 'población'], ['mente', 'tarea', 'preguntar él', 'cliente', 'amigo', 'familiar', 'opinión', 'frente', 'tecnología', 'opinión', 'recalcar

## Word Embedding

### Lemma Whitespace

In [6]:
import gensim
import numpy as np

'''
Here we train the Word2Vec model with a list of lists where each sublist is a tokenized paragraph.
After we get the word vectors per paragraph, we compute our paragraph meaning vector as the mean
of its word vectors.
'''

# Train Word2Vec model
lemmaModel = gensim.models.Word2Vec(tokenized_paragraphs_lemma_es, window=20, min_count=1, workers=4)
lemmaModel.save("paragraphModel")

# Calculate the meaning vector per paragraph
paragraph_vectors_lemma_es = []
for paragraph_tokens in tokenized_paragraphs_lemma_es:
    vectors = []
    for token in paragraph_tokens:
        if token in lemmaModel.wv.key_to_index:
            vectors.append(lemmaModel.wv[token])
    if len(vectors) > 0:
        paragraph_vectors_lemma_es.append(np.mean(vectors, axis=0))
    else:
        paragraph_vectors_lemma_es.append(np.zeros(lemmaModel.vector_size))

#print(paragraph_vectors_lemma[383])

#### Agregamos los vectores con su respectivo tokenization


In [7]:
df_esp['vector'] = paragraph_vectors_lemma_es


### PoS tagging

In [8]:
import nltk
from gensim.models import Word2Vec
import numpy as np

Creamos las tuplas con sus tags

In [9]:
nltk.download('averaged_perceptron_tagger')

tagged_sentences_esp = [nltk.pos_tag((paragraph)) for paragraph in tokenized_paragraphs_lemma_es]

# Entrenamiento del modelo Word2Vec
model_esp = Word2Vec(tagged_sentences_esp,  window=20, min_count=5, workers=4)

# Utilizar el modelo Word2Vec para encontrar palabras similares



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\afrincon\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
# Acceder a los vectores
vectors = model_esp.wv.vectors

# Acceder a las palabras
words = model_esp.wv.index_to_key
print(len(vectors))

223


In [11]:
print(len(words))

223


Creamos los vectores por cada parrafo

In [12]:
# Calculate the meaning vector per paragraph
paragraph_tag_vectors_lemma_es = []
vectors = model_esp.wv.vectors
# Acceder a las palabras
words = model_esp.wv.index_to_key
for paragraph_tokens in tagged_sentences_esp:
    vectors = []
    for tuples in paragraph_tokens:
        if tuples in model_esp.wv.index_to_key:
            ind=words.index(tuples)
            vectors.append(model_esp.wv[ind])
    if len(vectors) > 0:
        paragraph_tag_vectors_lemma_es.append(np.mean(vectors, axis=0))
    else:
        paragraph_tag_vectors_lemma_es.append(np.zeros(model_esp.vector_size))


In [13]:
df_esp['vector_tag'] = paragraph_tag_vectors_lemma_es

In [14]:
df_esp.head()

,index,Unnamed: 0,article_name,content,enumeration_in_article,file_id,language_code,vector,vector_tag
0,13,13,La inteligencia artificial en beneficio a la h...,Todos tenemos una opinión frente a lo que es i...,0,1,es,"[-0.0017099491, 0.0031091108, -0.0008322448, 0...","[-0.007439885, 0.007528569, 0.0012990527, 0.00..."
1,14,14,La inteligencia artificial en beneficio a la h...,"En contraste, dentro de nuestra realidad, pode...",1,1,es,"[-0.0006380161, 0.0014871386, 0.0013748091, 0....","[-0.0074089533, 0.007748193, 0.004417005, 0.00..."
2,15,15,La inteligencia artificial en beneficio a la h...,"Con esto en mente, junto a nuestro nos dimos ...",2,1,es,"[-0.0037842994, 0.0038468312, 0.00053745473, 0...","[-0.009978922, 0.0045816856, 0.00018485794, 0...."
3,16,16,La inteligencia artificial en beneficio a la h...,"En las ciencias de la computación, la AI es la...",3,1,es,"[-0.0026375998, 0.004141888, 0.00080027647, 0....","[-0.009559272, 0.0048803394, -0.0018194171, -0..."
4,17,17,La inteligencia artificial en beneficio a la h...,La inteligencia artificial débil (weak AI o Na...,4,1,es,"[-0.0035248038, 0.0058619226, 0.002862042, 9.1...","[-0.010966102, 0.0057403017, 0.0009402663, 0.0..."


## Similarity Function

In [15]:
import numpy as np
from gensim.models import KeyedVectors


def cosine_similarity_list(vectors_list, query_vector):
    #Compute the cosine similarity between the vector representation of the input and the vector representations of each sentence in the text
    similarity_scores = []
    for vector in vectors_list:
        score = query_vector.dot(vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
        similarity_scores.append(score)

    # Sort the sentences in descending order of their cosine similarity to the input and return the top-N most similar sentences
    n = 100
    most_similar_sentences = [[vectors_list[idx],idx] for idx in np.argsort(similarity_scores)[::-1][:n] if np.sum(vectors_list[idx]) != 0]

    return most_similar_sentences[:20]


In [16]:
cosine_similarity_list(df_esp['vector'],df_esp['vector'][0])

[[array([-1.7099491e-03,  3.1091108e-03, -8.3224481e-04,  1.5073832e-03,
          1.1896718e-03, -6.0802978e-03,  9.3697896e-04,  5.4561966e-03,
         -2.2524949e-03, -2.0218876e-03,  2.3691879e-04, -2.5431109e-03,
         -1.6291843e-03,  5.5836508e-04,  2.2634582e-03, -2.5537561e-03,
          2.1004879e-03, -4.4823168e-03, -6.5823662e-04, -4.9258596e-03,
          3.2517726e-03,  1.0604615e-03,  1.6069025e-03,  1.5955449e-04,
         -1.6676217e-03, -1.0679122e-03, -2.8572294e-03, -3.2606933e-03,
         -3.1231886e-03, -8.5862041e-05,  3.3593939e-03,  1.0759888e-03,
         -1.5849761e-03, -4.1302721e-04, -9.9655066e-04,  5.0146896e-03,
         -1.2482315e-03, -3.1911170e-03, -2.1793141e-03, -3.3233138e-03,
          2.2255988e-03, -1.1875086e-03, -6.5680937e-04,  1.0161740e-03,
          2.5780685e-03,  7.3667739e-05, -2.5788127e-03, -1.0037938e-03,
          7.2337873e-04,  8.9213881e-04,  1.6323990e-03, -3.0098709e-03,
         -3.1708051e-03,  1.3586576e-03, -3.2472974

## Prompt Preprocessing, Tokenization and Embedding

### Lemma with Pos-tagging

In [43]:
userPrompt="la inteligencia artificial es especial"
tokenized_prompt = preprocess_text_lemma(userPrompt)
tagged_prompt = nltk.pos_tag(tokenized_prompt)
print(tagged_prompt)
promptVector_lemma_eng = np.zeros((model_esp.vector_size,))
word_count  = 0
ind = 0
vectors = model_esp.wv.vectors
# Acceder a las palabras
words = model_esp.wv.index_to_key
palabras_ordenadas=sorted(words)
print(palabras_ordenadas)
print(tokenized_prompt)
for tag in tagged_prompt:
    if tag in model_esp.wv.index_to_key:
        ind=words.index(tag)
        print(tag)
        promptVector_lemma_eng += model_esp.wv[ind]
        word_count += 1

if word_count > 0:
    promptVector_lemma_eng /= word_count
    





[('inteligencia', 'JJ'), ('artificial', 'JJ'), ('especial', 'NN')]
[('2018', 'CD'), ('2019', 'CD'), ('2020', 'CD'), ('2021', 'CD'), ('actividad', 'NN'), ('alcance', 'NN'), ('algoritmo', 'NN'), ('ambiente', 'NN'), ('anteriormente', 'NN'), ('análisis', 'NN'), ('aplicación', 'NN'), ('aplicar', 'NN'), ('aprender', 'NN'), ('aprendizaje', 'NN'), ('arquitectura', 'NN'), ('artificial', 'JJ'), ('artículo', 'NN'), ('autónomo', 'NN'), ('avance', 'NN'), ('ayudar', 'NN'), ('año', 'NN'), ('basado', 'NN'), ('base', 'NN'), ('beneficio', 'NN'), ('buscar', 'NN'), ('campo', 'NN'), ('cantidad', 'NN'), ('capa', 'NN'), ('capacidad', 'NN'), ('capaz', 'NN'), ('característica', 'NN'), ('caso', 'NN'), ('ciencia', 'NN'), ('cliente', 'NN'), ('complejo', 'NN'), ('componente', 'NN'), ('comportamiento', 'NN'), ('computacional', 'JJ'), ('computador', 'NN'), ('computer', 'NN'), ('comunicación', 'NN'), ('concepto', 'NN'), ('confianza', 'NN'), ('conjunto', 'NN'), ('conocimiento', 'NN'), ('construcción', 'NN'), ('constru

### Testing PoS tagging

In [18]:
var = cosine_similarity_list(df_esp['vector_tag'],promptVector_lemma_eng)

C:\Users\afrincon\AppData\Local\Temp\ipykernel_19508\3599156865.py:9: RuntimeWarning: invalid value encountered in double_scalars
  score = query_vector.dot(vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))


## Similarity Test

In [19]:
var=np.array(var)
var[:,1]
possible_solutions=df_esp.iloc[var[:,1]]

C:\Users\afrincon\AppData\Local\Temp\ipykernel_19508\2543073143.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  var=np.array(var)


In [20]:
for paragraph in possible_solutions["content"]:
    print(paragraph, "\n")

Actualmente, Inteligencia Artificial es uno de los términos más utilizados en muchas áreas, no solo en entornos de tecnología pura, sino también en economía, leyes, medicina, entretenimiento, entre otros. Uno de los objetivos principales de la inteligencia artificial es mejorar el núcleo del negocio en el que está presente. Tradicionalmente, la mejora de un negocio se lleva a cabo por expertos, sin embargo, no siempre es rápida o eficiente. La Inteligencia Artificial (IA) permite en distintos escenarios delegar las tareas más complejas a una máquina, e increíblemente, sin decirle explícitamente cómo hacerlo. 

La inteligencia artificial débil (weak AI o Narrow AI en inglés), es una AI que lleva a cabo tareas totalmente específicas como jugar , dar recomendaciones como lo hace Netflix o poder hablar y dictarle comandos a nuestro celular a través de Siri. La inteligencia artificial general (AGI por sus siglas en inglés), es capaz de tener facultades cognitivas al mismo nivel que un ser h